# Reading Shakespeare DS

In [1]:
with open("shakespeare_input_ds.txt", "r", encoding="utf-8") as ds:
    text = ds.read()

# Check DS length

In [2]:
print("Length of text:", len(text))

Length of text: 1115394


# print first 100 characters

In [3]:
print("First 100 characters of text:", text[:100])

First 100 characters of text: First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


# Specify Unique Characters that appear in this text

In [4]:
chars_sorted = sorted(list(set(text)))
vocab_size = len(chars_sorted)
print("Vocabulary size:", vocab_size)
print("".join(chars_sorted))

Vocabulary size: 65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
